Hospital reception model
========================

1.  Apply Lindley's equation to make an approximation
2.  Model hospital as a M/G/1 Queue

In [1]:
import math
import numpy as np
import scipy as sp
#  Use scipy.stats because it includes the Erlang distribution
from scipy.stats import expon, erlang, t
import matplotlib.pyplot as plt

In [2]:
def lindley(m=55000, d = 5000):
    ''' Estimates waiting time with m customers, discarding the first d customers

        Lindley approximation for waiting time in a M/G/1 queue
    '''
    replications = 10
    lindley = []
    for rep in range(replications):
        y = 0
        SumY = 0
        for i in range(1, d):
            # Random number variable generation from scipy.stats
            # shape = 0, rate =1, 1 value
            a = expon.rvs(0, 1)
            # rate = .8/3, shape = 3
            x = erlang.rvs(3, scale = 0.8/3, size=1)
            y = max(0, y + x - a)
        for i in range(d, m):
            a = expon.rvs(0, 1)
            # rate = .8/3, shape = 3
            x = erlang.rvs(3, scale = 0.8/3, size=1)
            y = max(0, y + x - a)
            SumY += y
        result = SumY / (m - d)
        lindley.append(result)
    return lindley

In [3]:
np.random.seed(1234)
result = lindley()
print(result)
print(sp.mean(result))
print(sp.std(result))

[array([2.05799046]), array([2.15352756]), array([2.17230154]), array([2.33920724]), array([2.09431845]), array([2.13717148]), array([2.16830253]), array([2.10097151]), array([2.11777676]), array([2.22018782])]
2.156175535392981
0.0750746390816516


In [4]:
for i in range(len(result)):
    print ("%1d & %11.9f " % (i+1, result[i]))
print("average & %11.9f" % (sp.mean(result)))
print("std dev & %11.9f" % (sp.std(result)))

1 & 2.057990460 
2 & 2.153527560 
3 & 2.172301541 
4 & 2.339207242 
5 & 2.094318451 
6 & 2.137171478 
7 & 2.168302534 
8 & 2.100971509 
9 & 2.117776760 
10 & 2.220187818 
average & 2.156175535
std dev & 0.075074639


In [8]:
2.26 * sp.std(result) /math.sqrt(10)

0.053653949006963034

In [9]:
""" Discrete Event Simulation version """
#import SimPy.Simulation as Sim
import simpy
from random import seed

In [10]:
## Model components ------------------------

class Arrivals(object):
    """ Source generates customers randomly """

    def generate(self, env, meanTBA, resource):
        i=0
        while self.env.now() < G.maxTime:
            c = Patient(name="Patient%02d" % (i), sim=self.sim)
            self.env.process(c, c.visit(b=resource))
            t = expon.rvs(0, 1.0 / meanTBA, size = 1)
            yield Sim.hold, self, t
            i+=1

class Patient(object):
    """ Patient arrives, is served and leaves """

    def visit(self,  b):
        arrive = self.env.now()
        yield simpy.request, self, b
        wait = self.env.now() - arrive
        tib = erlang.rvs(G.phases, scale = float(G.timeReceptionist)/G.phases, size = 1)  
        yield simpy.hold, self, tib                            #2
        yield simpy.release, self, b


In [11]:
## Experiment data -------------------------
class G:
    maxTime = 55000.0    # minutes
    warmuptime = 5000.0
    timeReceptionist = 0.8  # mean, minutes
    phases = 3
    ARRint = 1.0      # mean, minutes
    theseed = 99999 

In [12]:
## Model/Experiment ------------------------------
class Hospitalsim(simpy.Environment):
    def run(self, theseed):
        np.random.seed(theseed)
        self.receptionist = Sim.Resource(name="Reception", capacity = 1, unitName="Receptionist", monitored=True, sim=self)
        s = Arrivals('Source', sim=self)
        self.initialize()
        self.activate(s, s.generate(meanTBA=G.ARRint, resource=self.receptionist), at=0.0)
        self.simulate(until=G.maxTime)
        avgutilization = self.receptionist.actMon.timeAverage()[0]
        avgwait = self.receptionist.waitMon.mean()
        avgqueue = self.receptionist.waitMon.timeAverage()[0]
        leftinqueue= mg1.receptionist.waitMon.yseries()[-1:][0]
        return [avgwait, avgqueue, leftinqueue,avgutilization]

In [14]:
reps=10
hospitalreception = []
for i in range(reps):
    mg1 = Hospitalsim()
    #mg1.startCollection(when=G.warmuptime, monitors=mg1.allMonitors)
    result = mg1.run(4321 + i)
    hospitalreception.append(result)

NameError: name 'Sim' is not defined

In [ ]:
plt.plot(mg1.receptionist.waitMon.tseries(), mg1.receptionist.waitMon.yseries())

In [ ]:
hospitalaverage = mean(hospitalreception, axis=0)
hospitalstd = std(hospitalreception, axis=0)
hospital95 = [hstd * t.ppf(0.975, reps-1) for hstd in hospitalstd]

In [ ]:
for i in range(len(hospitalreception)):
    print ("%1d & %11.9f  & %11.9f  & %4d  & %11.9f " % (i+1, hospitalreception[i][0], hospitalreception[i][1], hospitalreception[i][2], hospitalreception[i][3]))
print("average & %11.9f  & %11.9f  & %11.9f  & %11.9f " % (hospitalaverage[0], hospitalaverage[1], hospitalaverage[2], hospitalaverage[3]))
print("std dev & %11.9f  & %11.9f  & %11.9f  & %11.9f " % (hospitalstd[0], hospitalstd[1], hospitalstd[2], hospitalstd[3]))
print("95 C.I. & %11.9f  & %11.9f  & %11.9f  & %11.9f " % (hospital95[0], hospital95[1], hospital95[2], hospital95[3]))

In [ ]:
## Experiment data -------------------------
class G:
    maxTime = 10 * 60.0    # minutes
    warmuptime = 0.0
    timeReceptionist = 0.6  # mean, minutes
    phases = 4
    timeKiosk = 0.8 # mean minutes
    phasesKiosk = 3
    ARRint = 1.0      # mean, minutes
    theseed = 99999 

In [ ]:
## Model/Experiment ------------------------------
class HospitalKiosksim(Sim.Simulation):
    # modified for use on midterm
    def run(self, theseed, timeKiosk):
        np.random.seed(theseed)
        self.timeService = timeKiosk
        self.phases = G.phasesKiosk
        self.receptionists = 1
        self.receptionist = Sim.Resource(name="Reception", unitName="Kiosk", monitored=True, sim=self)
        s = Arrivals('Source', sim=self)
        self.initialize()
        self.activate(s, s.generate(meanTBA=G.ARRint, resource=self.receptionist), at=0.0)
        self.simulate(until=G.maxTime)
        avgutilization = self.receptionist.actMon.timeAverage()[0]
        avgwait = self.receptionist.waitMon.mean()
        avgqueue = self.receptionist.waitMon.timeAverage()[0]
        leftinqueue= mg1.receptionist.waitMon.yseries()[-1:][0]
        return [avgwait]

In [ ]:
reps=50
khospitalreception = []
timeKioskrange = [0.6, 0.75, 0.775, 0.8, 0.825,]
for j in timeKioskrange:
    hospitalreception = []
    for i in range(reps):
        mg1 = HospitalKiosksim()
        mg1.startCollection(when=G.warmuptime, monitors=mg1.allMonitors)
        result = mg1.run(4321 + i, j)
        hospitalreception.append(result)
    khospitalreception.append(hospitalreception)

In [ ]:
hospitalaverage = mean(khospitalreception, axis=1)
hospitalstd = std(khospitalreception, axis=0)
hospital95 = [hstd * t.ppf(0.975, reps-1) for hstd in hospitalstd]

In [ ]:
hospitalaverage

In [ ]:
sshospitalwait = [sum([math.pow(khospitalreception[j][i] -hospitalaverage[j],2) for i in range(len(khospitalreception[j]))]) for j in range(len(khospitalreception))]

In [ ]:
meanexperiment = mean(khospitalreception[1:5])
meanoptions = [mean(khospitalreception[i]) for i in range(1,5)]

In [ ]:
ssexperiment = sum([math.pow(meanoptions[i] -meanexperiment,2) for i in range(len(meanoptions))])

In [ ]:
ssexperiment

In [ ]:
[sum([math.pow(khospitalreception[j][i] -meanexperiment,2) for i in range(len(khospitalreception[j]))]) for j in range(len(khospitalreception))]

In [ ]:
print(sigma2T)
print(sigma2S)
print(sigma2I)
sigmaI_sigmaS = sigma2I/(sigma2S/n)
print(sigmaI_sigmaS)

In [ ]:
sum([math.pow(hospitalaverage[i] - meanexperiment,2) \
                         for i in range(1,len(timeKioskrange))])

In [ ]:
[math.pow(hospitalaverage[i] - meanexperiment,2) \
                         for i in range(1,len(timeKioskrange))]

In [ ]:
[sum([math.pow(khospitalreception[i][j] - hospitalaverage[i],2)
                                for j in range(reps)]) for i in range(1,len(timeKioskrange))]

In [ ]:
[sum([math.pow(khospitalreception[i][j] - hospitalaverage[i],2)
                                for j in range(reps)]) for i in range(1,len(timeKioskrange))]